In [8]:
import pandas as pd
import numpy as np
import re
import string
import time
from datetime import datetime
from datetime import date
import os

#Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# ML Libraries
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import metrics

# Save model
import pickle
import joblib

In [9]:
# directorDFfileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\cumulativeDirectorDF_Mar-21-2020.csv'
# moviesDataFileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\OnlyMovieDirectorDF.csv'
moviesDataFileLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\2_Movies_WithAwards.csv'

# storedDF = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\StoreDF_Mar-25-2020.csv'
# storeModelLocation = 'C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\Classification Models\\SVM'

saveFileToPath = "C:\\Yuva\\ITU\\4th Sem\\Thesis\\Data\\"

data = pd.read_csv(moviesDataFileLocation)
# storeDF = pd.read_csv(storedDF)

In [10]:
storeDF_col_names =  ['run_model','features_selected', 'record_count', 'male_count','female_count','predicting','Order_ABS','Order_Pos_Neg',
              'TN','FP','FN','TP','total_time']
# storeDF  = pd.DataFrame(columns = storeDF_col_names)

In [11]:
# Helper method for save_to_file

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [12]:
# Function: Save Dataframe to CSV

def save_to_file(dataFrame, folderPath):
    today = date.today()
    todayDate = today.strftime("%b-%d-%Y")
#     dataFrame.to_csv(folderPath+ '\\' + get_df_name(dataFrame) + '_' + todayDate  + '.csv',index=False, encoding="utf8" )
    dataFrame.to_csv(folderPath+ '\\' + get_df_name(dataFrame) + '_' + todayDate  + '.csv',index=False, encoding="utf-8-sig" )

In [13]:
director_data.head()

,director_ids,director_names,movie_id,title,review_count_user,review_count_critic,metascore,rating_value,rating_count,release_date,...,Winner_Actor,Winner_Actor in a Leading Role,Winner_Actor in a Supporting Role,Winner_Actress,Winner_Actress in a Leading Role,Winner_Actress in a Supporting Role,Winner_Assistant Director,Winner_Directing,Winner_Directing (Comedy Picture),Winner_Directing (Dramatic Picture)
0,nm0000485,Fritz Lang,tt0017136,Metropolis (1927) - IMDb,418,263,98,8.3,114917,5 March 1927 (Denmark) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,nm0000122,Charles Chaplin,tt0027977,Les temps modernes (1936) - IMDb,213,119,96,8.6,147990,24 September 1936 (France) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,nm0281808,Victor Fleming,tt0032138,"Óz, a csodák csodája (1939) - IMDb",540,213,100,8.1,299644,21 March 1940 (Hungary) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,nm0002030,George Cukor,tt0032138,"Óz, a csodák csodája (1939) - IMDb",540,213,100,8.1,299644,21 March 1940 (Hungary) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,nm0003506,James Mangold,tt0035423,Kate & Leopold (2001) - IMDb,317,124,44,6.4,65765,25 December 2001 (USA) See more »,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
director_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3709 entries, 0 to 3708
Columns: 142 entries, director_ids to Winner_Directing (Dramatic Picture)
dtypes: float64(27), int64(107), object(8)
memory usage: 4.0+ MB


In [ ]:
required_features_director_data = ['movies_produced',
                                   'Average_rating_value',
                                   'Average_rating_count',
                                   'career_Length',
                                   'Gender',
                                   'birthYear',
                                   'CareerStartYear',
                                   'average_movie_per_year',
                                   'median_inter_event_time',
                                   'average_inter_event_time'
                                  ]

In [ ]:
director_data = director_data[required_features_director_data]
print('The shape of our Director specific dataset is:', director_data.shape)

In [ ]:
# Generic function to equally max split the dataframe with respect to the column Name, in our case the Gender column.

def max_equal_split_on_columnName(df,columnName='Gender'):
    df1 = pd.DataFrame()
    for i in df[columnName].value_counts().index:
        df1 = df1.append(df[(df[columnName]==i)].sample(n=df[columnName].value_counts().values.min(),replace = False))        
    return df1.reset_index(drop=True)

In [ ]:
# Generic function to equally split the dataframe with respect to the column Name, in our case the Gender column.

def equal_split_on_columnName(nSampleEach, df,columnName='Gender'):
    df1 = pd.DataFrame()
    for i in df[columnName].value_counts().index:
        df1 = df1.append(df[(df[columnName]==i)].sample(n=nSampleEach,replace = False))        
    return df1.reset_index(drop=True)

In [ ]:
def print_feature_importances(coef, names, top=-1):
    whole = []
    imp = coef
    imp,names = zip(*sorted(list(zip(imp, names)),reverse=True))
    for i in range(len(names)):
        string = "{} :: {}".format(str(names[i]),str(round(imp[i],3)))
        whole.append(string)
    return whole

In [ ]:
def save_classifier_model(file_name,model):
    path = storeModelLocation + '\\' + str(file_name) + '.sav'
    pickle.dump(model, open(path, 'wb'))

In [ ]:
def train_model(X_train, X_test, y_train, y_test,kernelType,gamma_parameter):
    
    # Training the model
#     svclassifier = SVC(kernel=kernelType,gamma = gamma_parameter)
    if(kernelType == 'linear'):
        svclassifier = SVC(kernel=kernelType,gamma = gamma_parameter,probability=True)
    elif(kernelType == 'poly'):
        svclassifier = SVC(kernel=kernelType,gamma = gamma_parameter,probability=True,degree=8)
    svclassifier.fit(X_train, y_train)
    
    # Prediction based on the model trained
    y_pred = svclassifier.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
        
#     print("\nClassification Report\n")
#     print(classification_report(y_test, y_pred))

    return y_pred,tn, fp, fn, tp,svclassifier

In [ ]:
def plot_roc_auc_curve(model, X_test, y_test):
    # calculate the fpr and tpr for all thresholds of the classification
#     X_test = X_test.replace(['Female','Male'],[0,1])
    probabilities = model.predict_proba(np.array(X_test))
    predictions = probabilities[:, 1]
    fpr, tpr, threshold = metrics.roc_curve(y_test, predictions,pos_label = 'Male')
#     roc_auc = metrics.auc(fpr, tpr)
    roc_auc = metrics.roc_auc_score(fpr, tpr)
    print("\nThe Area under the curve - ROC is :" + str(roc_auc))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
    plt.legend(loc='lower right')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    
    return roc_auc

In [ ]:
# def plot_roc(model, X_test, y_test):
#     # calculate the fpr and tpr for all thresholds of the classification
#     probabilities = model.predict_proba(np.array(X_test))
#     predictions = probabilities[:, 1]
#     fpr, tpr, threshold = metrics.roc_curve(y_test, predictions,pos_label = 'Male')
#     roc_auc = metrics.auc(fpr, tpr)
#     print("\nThe Area under the curve - ROC is :" + str(roc_auc))
#     plt.title('Receiver Operating Characteristic')
#     plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % roc_auc)
#     plt.legend(loc='lower right')
#     plt.plot([0, 1], [0, 1], 'r--')
#     plt.xlim([0, 1])
#     plt.ylim([0, 1])
#     plt.ylabel('True Positive Rate')
#     plt.xlabel('False Positive Rate')
#     plt.show()
    
#     return roc_auc

In [ ]:
def run_model_SVM(model_comments,df,storeDF,featuresSelected,predictingFeature="Gender",kernelType='linear',gamma='auto'):
    
    start = time.time()
    print("Started Reading files at :" + str(datetime.utcfromtimestamp(start).strftime('%Y-%m-%d %H:%M:%S')))
    record_count = len(df)
    maleCount = df[predictingFeature].value_counts()["Male"]
    femaleCount = df[predictingFeature].value_counts()["Female"]
    
    # divide the data into attributes and labels
    X = df[featuresSelected]
    y = df[predictingFeature]
    
    # Divide the data into training and test sets
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
    y_pred,tn, fp, fn, tp,trainedSVClassifier = train_model(X_train, X_test, y_train, y_test,kernelType,gamma)
    
    # Save model to a path
#     today = date.today()
#     todayDate = today.strftime("%b_%d_%Y")
#     save_model_name = 'SVC' + "_" + str(len(featuresSelected)) + 'features_' + df + "_" + kernelType + '_' + gamma + '_' + todayDate 
    save_model_name = 'SVC' + "_" + str(len(featuresSelected)) + 'features_' + str(get_df_name(df)) + "_" + kernelType + '_' + gamma 
    save_classifier_model(save_model_name,trainedSVClassifier)
    
    # Area under curve - ROC
#     auc = plot_roc(trainedSVClassifier,X_test,y_test)
#     roc_auc = plot_roc_auc_curve(trainedSVClassifier,X_train, X_test, y_train, y_test)

    
    # Feature importance based on contribution to the classifier model
    print("\nAbsolute contributions\n")
    absoluteListOrder = print_feature_importances(abs(trainedSVClassifier.coef_[0]), featuresSelected)
    for j in absoluteListOrder:
        print(j)
    
    print("\nPositive and Negative contributions\n")
    listOrder = print_feature_importances((trainedSVClassifier.coef_[0]), featuresSelected)
    for i in listOrder:
        print(i)
    
    # Stop timer
    end = time.time()
    print("\nStopped Reading files at :" + str(datetime.utcfromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S')))
    total_time = str(round((end - start)/60,2))
    print("Total time taken in minutes: " + str((end - start)/60))
    
    return y_test,y_pred,trainedSVClassifier,[model_comments,featuresSelected,record_count,maleCount,femaleCount,predictingFeature,absoluteListOrder,listOrder,tn, fp, fn, tp,total_time]
    
    

# New attempt

In [ ]:
# # divide the data into attributes and labels
# X = equalDF[model_features]
# y = equalDF['Gender']

# # Divide the data into training and test sets

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state = 42 )

# # Training the model
# svclassifier = SVC(kernel='linear',probability=True)
# svclassifier.fit(X_train, y_train)

# # Prediction based on the model trained
# y_pred = svclassifier.predict(X_test)

# tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()

In [ ]:
def evaluation_analysis(true_label,predicted): 
    '''
    return all metrics results
    '''
    true_label = true_label.replace(['Female','Male'],[0,1])
    predicted = [p.replace('Female','0') for p in predicted]
    predicted = [p.replace('Male','1') for p in predicted]
    predicted = [int(i) for i in predicted]
    
    print( "Accuracy",round(metrics.accuracy_score(true_label, predicted),3))
    print( "F1 score macro",round(metrics.f1_score(true_label, predicted, average='macro'),3))    
    print( "F1 score micro",round(metrics.f1_score(true_label, predicted, average='micro') ,3))
    print( "F1 score average",round(metrics.f1_score(true_label, predicted, average='weighted'),3))
    print( "Precision score",round(metrics.precision_score(true_label, predicted, average='macro'),3))
    print( "Recall score",round(metrics.recall_score(true_label, predicted, average='macro'),3))
    print( "AUC&ROC",round(metrics.roc_auc_score(true_label, predicted),3))
    
    print( "Hamming_loss",round(metrics.hamming_loss(true_label, predicted),3))
    print( "Jaccard_similarity_score",round( metrics.jaccard_similarity_score(true_label, predicted),3))
    print( "Zero_one_loss", round(metrics.zero_one_loss(true_label, predicted),3))
    print( "Matthews_corrcoef", round(metrics.matthews_corrcoef(true_label, predicted),3))
    print( "log_loss", round(metrics.log_loss(true_label, predicted),3))

    print("\nClassification Report\n")
    print(classification_report(true_label, predicted))


In [ ]:
# # Convert test class and predicted class to integers and print metrics
# test_y = y_test.replace(['Female','Male'],[0,1])

# pred_y = [p.replace('Female','0') for p in y_pred];
# pred_y = [p.replace('Male','1') for p in pred_y];
# pred_y = [int(i) for i in pred_y]

# evaluation_analysis(test_y,pred_y)

## Running Director Dataset - with model features

In [ ]:
# Run 1- Director Dataset - Equal and All data - Linear SVM
model_features = ['movies_produced','Average_rating_value','Average_rating_count',
                  'career_Length','birthYear','CareerStartYear','average_movie_per_year',
                  'median_inter_event_time','average_inter_event_time']
model_comments = 'Run 1a - All data - Linear SVM - Running with all features'
# model_comments = 'Run 1b - Equal data - Linear SVM - Running with all features'

# # Run 2 - Director Dataset - Equal and All data - Linear SVM
# model_features = ['career_Length','average_movie_per_year',
#                   'median_inter_event_time','CareerStartYear',
#                   'average_inter_event_time']
# model_comments = 'Run 2a - All data - Linear SVM - Running with top 5 absolute contribution features from Run 1a : career_Length , average_movie_per_year , median_inter_event_time , CareerStartYear , average_inter_event_time'
# # model_comments = 'Run 2b - Equal Data - Linear SVM - Running with top 5 absolute contribution features from Run 1a : career_Length , average_movie_per_year , median_inter_event_time , CareerStartYear , average_inter_event_time'

# # Run 3 - Director Dataset - Equal and All data - Linear SVM
# model_features = ['average_inter_event_time','Average_rating_value',
#                   'median_inter_event_time','average_movie_per_year',
#                   'CareerStartYear']
# model_comments = 'Run 3a - All data - Linear SVM - Running with top 5 absolute contribution features from Run 1b : average_inter_event_time,Average_rating_value,median_inter_event_time,average_movie_per_year,CareerStartYear'
# # model_comments = 'Run 3b - Equal data - Linear SVM - Running with top 5 absolute contribution features from Run 1b : average_inter_event_time,Average_rating_value,median_inter_event_time,average_movie_per_year,CareerStartYear'

# # Run 4 - Compare results from Random Forest featrues - Equal and All data - Linear SVM
# model_features = ['Average_rating_value','Average_rating_count',
#                   'birthYear','CareerStartYear']
# # model_comments = 'Run 4a - All data - Linear SVM - Running with top 5 contribution features from Random Forest : Average_rating_value,Average_rating_count,birthYear,CareerStartYear'
# model_comments = 'Run 4b - Equal data - Linear SVM - Running with top 5 contribution features from Random Forest : Average_rating_value,Average_rating_count,birthYear,CareerStartYear'




In [ ]:
temp_director_only = director_data.dropna(subset=model_features)
temp_director_only.info()

In [ ]:
equalDF = equal_split_on_columnName(100,temp_director_only)

In [ ]:
maxEqualDF = max_equal_split_on_columnName(temp_director_only)

In [ ]:
def print_features_selected(df):
    print('[',end = '')
    for i,v in enumerate(df.columns.values):
        if(i==len(df.columns.values)-1):
            print("'" + str(v) +"']")
        else:
            print("'" + str(v) +"',")

In [ ]:
# print_features_selected(temp_director_only)
print_features_selected(maxEqualDF[model_features])

## Running Whole Director DF 

In [ ]:
y_test,y_pred,model_SVC,storeDF.loc[len(storeDF)] = run_model_SVM(model_comments,temp_director_only,storeDF,
                                                                  model_features,'Gender','linear','auto')

In [ ]:
evaluation_analysis(y_test,y_pred)

In [ ]:
storeDF

In [ ]:
# #Save a copy of true and predicted for Manual intervention
# Actual = y_test.to_frame()
# Actual['predicter'] = y_pred
# save_to_file(Actual,saveFileToPath)

## Running Balanced max Equal DF 

In [ ]:
y_test,y_pred,model_SVC,storeDF.loc[len(storeDF)] = run_model_SVM(model_comments,maxEqualDF,storeDF,
                                                                  model_features,'Gender','linear','auto')

In [ ]:
evaluation_analysis(y_test,y_pred)

In [ ]:
storeDF

## Test Run sample DF

In [ ]:
y_test,y_pred,model_SVC,storeDF.loc[len(storeDF)] = run_model_SVM(model_comments,equalDF,storeDF,
                                                                  model_features,'Gender','poly','auto')

In [ ]:
evaluation_analysis(y_test,y_pred)

In [ ]:
evaluation_analysis(y_test,y_pred)

In [ ]:
storeDF

In [ ]:
save_to_file(storeDF,saveFileToPath)